In [ ]:
import csv
import numpy as np
import pandas as pd

df = pd.read_csv('C:/Users/aasth/Desktop/Engg of Big Data/Final Project/Archive/yelp_academic_dataset_business.csv')

In [6]:
import nltk
nltk.download()

showing info http://www.nltk.org/nltk_data/


True

In [7]:

class Splitter(object):
    def __init__(self):
        self.nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self, text):
        """
        input format: a paragraph of text
        output format: a list of lists of words.
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        """
        sentences = self.nltk_splitter.tokenize(text)
        tokenized_sentences = [self.nltk_tokenizer.tokenize(sent) for sent in sentences]
        return tokenized_sentences


class POSTagger(object):
    def __init__(self):
        pass
        
    def pos_tag(self, sentences):
        """
        input format: list of lists of words
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        output format: list of lists of tagged tokens. Each tagged tokens has a
        form, a lemma, and a list of tags
            e.g: [[('this', 'this', ['DT']), ('is', 'be', ['VB']), ('a', 'a', ['DT']), ('sentence', 'sentence', ['NN'])],
                    [('this', 'this', ['DT']), ('is', 'be', ['VB']), ('another', 'another', ['DT']), ('one', 'one', ['CARD'])]]
        """

        pos = [nltk.pos_tag(sentence) for sentence in sentences]
        #adapt format
        pos = [[(word, word, [postag]) for (word, postag) in sentence] for sentence in pos]
        return pos


In [23]:
text = """Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road."""

splitter = Splitter()
postagger = POSTagger()

splitted_sentences = splitter.split(text)

print(splitted_sentences)

pos_tagged_sentences = postagger.pos_tag(splitted_sentences)

print(pos_tagged_sentences)


[['Mr', 'Hoagie', 'is', 'an', 'institution', '.'], ['Walking', 'in', ',', 'it', 'does', 'seem', 'like', 'a', 'throwback', 'to', '30', 'years', 'ago', ',', 'old', 'fashioned', 'menu', 'board', ',', 'booths', 'out', 'of', 'the', '70s', ',', 'and', 'a', 'large', 'selection', 'of', 'food', '.'], ['Their', 'speciality', 'is', 'the', 'Italian', 'Hoagie', ',', 'and', 'it', 'is', 'voted', 'the', 'best', 'in', 'the', 'area', 'year', 'after', 'year', '.'], ['I', 'usually', 'order', 'the', 'burger', ',', 'while', 'the', 'patties', 'are', 'obviously', 'cooked', 'from', 'frozen', ',', 'all', 'of', 'the', 'other', 'ingredients', 'are', 'very', 'fresh', '.'], ['Overall', ',', 'its', 'a', 'good', 'alternative', 'to', 'Subway', ',', 'which', 'is', 'down', 'the', 'road', '.']]
[[('Mr', 'Mr', ['NNP']), ('Hoagie', 'Hoagie', ['NNP']), ('is', 'is', ['VBZ']), ('an', 'an', ['DT']), ('institution', 'institution', ['NN']), ('.', '.', ['.'])], [('Walking', 'Walking', ['VBG']), ('in', 'in', ['IN']), (',', ',', ['

In [24]:
class DictionaryTagger(object):
    def __init__(self, dictionary_paths):
        files = [open(path, 'r') for path in dictionary_paths]
        dictionaries = [yaml.load(dict_file) for dict_file in files]
        map(lambda x: x.close(), files)
        self.dictionary = {}
        self.max_key_size = 0
        for curr_dict in dictionaries:
            for key in curr_dict:
                if key in self.dictionary:
                    self.dictionary[key].extend(curr_dict[key])
                else:
                    self.dictionary[key] = curr_dict[key]
                    self.max_key_size = max(self.max_key_size, len(key))

    def tag(self, postagged_sentences):
        return [self.tag_sentence(sentence) for sentence in postagged_sentences]

    def tag_sentence(self, sentence, tag_with_lemmas=False):
        """
        the result is only one tagging of all the possible ones.
        The resulting tagging is determined by these two priority rules:
            - longest matches have higher priority
            - search is made from left to right
        """
        tag_sentence = []
        N = len(sentence)
        if self.max_key_size == 0:
            self.max_key_size = N
        i = 0
        while (i < N):
            j = min(i + self.max_key_size, N) #avoid overflow
            tagged = False
            while (j > i):
                expression_form = ' '.join([word[0] for word in sentence[i:j]]).lower()
                expression_lemma = ' '.join([word[1] for word in sentence[i:j]]).lower()
                if tag_with_lemmas:
                    literal = expression_lemma
                else:
                    literal = expression_form
                if literal in self.dictionary:
                    #self.logger.debug("found: %s" % literal)
                    is_single_token = j - i == 1
                    original_position = i
                    i = j
                    taggings = [tag for tag in self.dictionary[literal]]
                    tagged_expression = (expression_form, expression_lemma, taggings)
                    if is_single_token: #if the tagged literal is a single token, conserve its previous taggings:
                        original_token_tagging = sentence[original_position][2]
                        tagged_expression[2].extend(original_token_tagging)
                    tag_sentence.append(tagged_expression)
                    tagged = True
                else:
                    j = j - 1
            if not tagged:
                tag_sentence.append(sentence[i])
                i += 1
        return tag_sentence


In [13]:
import yaml

global yaml

In [25]:
dicttagger = DictionaryTagger([ 'C:/Users/aasth/Desktop/dicts/positive.yml', 'C:/Users/aasth/Desktop/dicts/negative.yml'])

dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)

print(dict_tagged_sentences)



[[('Mr', 'Mr', ['NNP']), ('Hoagie', 'Hoagie', ['NNP']), ('is', 'is', ['VBZ']), ('an', 'an', ['DT']), ('institution', 'institution', ['NN']), ('.', '.', ['.'])], [('Walking', 'Walking', ['VBG']), ('in', 'in', ['IN']), (',', ',', [',']), ('it', 'it', ['PRP']), ('does', 'does', ['VBZ']), ('seem', 'seem', ['VB']), ('like', 'like', ['IN']), ('a', 'a', ['DT']), ('throwback', 'throwback', ['NN']), ('to', 'to', ['TO']), ('30', '30', ['CD']), ('years', 'years', ['NNS']), ('ago', 'ago', ['RB']), (',', ',', [',']), ('old', 'old', ['JJ']), ('fashioned', 'fashioned', ['VBN']), ('menu', 'menu', ['JJS']), ('board', 'board', ['NN']), (',', ',', [',']), ('booths', 'booths', ['VBZ']), ('out', 'out', ['IN']), ('of', 'of', ['IN']), ('the', 'the', ['DT']), ('70s', '70s', ['CD']), (',', ',', [',']), ('and', 'and', ['CC']), ('a', 'a', ['DT']), ('large', 'large', ['JJ']), ('selection', 'selection', ['NN']), ('of', 'of', ['IN']), ('food', 'food', ['NN']), ('.', '.', ['.'])], [('Their', 'Their', ['PRP$']), ('sp

In [27]:
#!/usr/bin/python 
#
# (originally entered at https://gist.github.com/1035399)
#
# License: GPLv3
#
# To download the AFINN word list do:
# wget http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/6010/zip/imm6010.zip
# unzip imm6010.zip
#
# Note that for pedagogic reasons there is a UNICODE/UTF-8 error in the code.

import math
import re
import sys
reload(sys)


# AFINN-111 is as of June 2011 the most recent version of AFINN
filenameAFINN = 'AFINN/AFINN-111.txt'
afinn = dict(map(lambda (w, s): (w, int(s)), [ 
            ws.strip().split('\t') for ws in open(filenameAFINN) ]))

# Word splitter pattern
pattern_split = re.compile(r"\W+")

def sentiment(text):
    """
    Returns a float for sentiment strength based on the input text.
    Positive values are positive valence, negative value are negative valence. 
    """
    words = pattern_split.split(text.lower())
    sentiments = map(lambda word: afinn.get(word, 0), words)
    if sentiments:
        # How should you weight the individual word sentiments? 
        # You could do N, sqrt(N) or 1 for example. Here I use sqrt(N)
        sentiment = float(sum(sentiments))/math.sqrt(len(sentiments))
        
    else:
        sentiment = 0
    return sentiment



if __name__ == '__main__':
    # Single sentence example:
    text = "Finn is stupid and idiotic"
    print("%6.2f %s" % (sentiment(text), text))
    
    # No negation and booster words handled in this approach
    text = "Finn is only a tiny bit stupid and not idiotic"
    print("%6.2f %s" % (sentiment(text), text))



SyntaxError: invalid syntax (<ipython-input-27-028744a2747d>, line 21)

In [16]:
def value_of(sentiment):
    if sentiment == 'positive': return 1
    if sentiment == 'negative': return -1
    return 0

def sentiment_score(review):    
    return sum ([value_of(tag) for sentence in dict_tagged_sentences for token in sentence for tag in token[2]])



In [26]:
sentiment_score(dict_tagged_sentences)

1